In [2]:
import pandas as pd
import os 

input_data_dir = os.getcwd().replace('/notebooks', '') + '/data/external'

team_filename = 'game_summary_by_team.csv'
indiv_player_game_stats_filename = 'game_summary_by_player.csv'
indiv_player_metrics_filename = 'player_metadata.csv'

team_df = pd.read_csv(os.path.join(input_data_dir, team_filename))
indiv_player_game_df = pd.read_csv(os.path.join(input_data_dir, indiv_player_game_stats_filename))
indiv_player_metrics_df = pd.read_csv(os.path.join(input_data_dir, indiv_player_metrics_filename))

first_cols = ['player_link', 'player_name', 'season', 'week', 'team','position']
other_cols = [x for x in list(indiv_player_game_df.columns) if x not in first_cols]
indiv_player_game_df_cols = first_cols + other_cols
indiv_player_game_df = indiv_player_game_df[indiv_player_game_df_cols]
indiv_player_game_df.head()

unique_games = team_df["url"].unique()

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
def get_player_stats(players_df, team, opp_flg, off_snaps, def_snaps):
    col_prepend = "opp" if opp_flg == True else "team"
    pos_stats_dict = {
        "QB" : ["fumbles", "pass_att", "pass_cmp", "pass_int", "pass_sacked", "pass_yds", 
                "rush_att", "rush_long", "rush_td", "rush_yds", "snaps"],
        "RB" : ["fumbles", "rush_att", "rush_long", "rush_td", "rush_yds",
                "rec", "rec_long", "rec_td", "rec_yds", "targets", "snaps"],
        "WR" : ["fumbles", "rec", "rec_long", "rec_td", "rec_yds", "targets", "snaps"],
        "TE" : ["fumbles", "rec", "rec_long", "rec_td", "rec_yds", "targets", "snaps"],
        "OL" : [],
        "DL" : ["fumbles_forced", "sacks", "tackles_assists", "tackles_solo", "snaps"],
        "LB" : ["fumbles_forced", "sacks", "tackles_assists", "tackles_solo", "snaps"],
        "DB" : ["def_int", "fumbles_forced", "sacks", "tackles_assists", "tackles_solo", "snaps"],
        "K" : ["fga", "fgm", "xpa", "xpm"],
        "P" : ["punt", "punt_yds_per_punt"]
    }
    team_players_df = players_df[players_df["team"] == team]
    pos_player_dict = {}
    
    for pos in team_players_df["position"].unique():
        pos_colname = pos.lower()
        pos_players_df = team_players_df[team_players_df["position"] == pos].fillna(0)
        pos_stats = pos_stats_dict[pos]
        
        for stat in pos_stats:
            pos_aggregated_stat = pos_players_df[stat].sum()
            pos_player_dict["{}_{}_{}_sum".format(col_prepend, pos_colname, stat)] = pos_aggregated_stat
        
        pos_player_avg_dict = get_pos_player_avg_stats(pos_players_df, pos, col_prepend, off_snaps, def_snaps)
        pos_player_dict.update(pos_player_avg_dict)
    return pos_player_dict

def get_pos_player_avg_stats(pos_players_df, pos, col_prepend, off_snaps, def_snaps):
    
    numerator_key = "numerator"
    denominator_key = "denominator"
    pos_colname = pos.lower()

    avg_stats_def = {
        "yds_per_rec" : {numerator_key : "rec_yds", denominator_key : "rec"},
        "yds_per_rush" : {numerator_key : "rush_yds", denominator_key : "rush_att"},
        "yds_per_target" : {numerator_key: "rec_yds", denominator_key : "targets"},
        "yds_per_snap" : {numerator_key : ["rush_yds", "rec_yds"], denominator_key : None, "denominator_val": off_snaps },
        "yds_per_pass_att" : {numerator_key : "pass_att", denominator_key : "pass_yds"},
        "yds_per_pass_comp" : {numerator_key : "pass_cmp", denominator_key : "pass_yds"},
        "tackles_per_snap" : {numerator_key : ["tackles_assists", "tackles_solo"], denominator_key :  None, "denominator_val": def_snaps},
        "fg_pct" : {numerator_key : "fgm", denominator_key : "fga"}
    }
    pos_stats_dict = {
        "QB" : ["yds_per_pass_att", "yds_per_pass_comp"],
        "RB" : ["yds_per_rush", "yds_per_rec", "yds_per_target", "yds_per_snap"],
        "WR" : ["yds_per_rec", "yds_per_target", "yds_per_snap"],
        "TE" : ["yds_per_rec", "yds_per_target", "yds_per_snap"],
        "OL" : [],
        "DL" : ["tackles_per_snap"],
        "LB" : ["tackles_per_snap"],
        "DB" : ["tackles_per_snap"],
        "P" : [],
        "K" : ["fg_pct"]
    }
    pos_stat_list = pos_stats_dict[pos]
    
    pos_player_avg_dict = {}
    for stat in pos_stat_list:
        stat_config = avg_stats_def[stat]
        numerator = stat_config[numerator_key]
        denominator = stat_config[denominator_key]
        
        denominator_val = pos_players_df[denominator].sum() if denominator is not None else stat_config["denominator_val"]
        if isinstance(numerator, list):
            numerator_val = pos_players_df[numerator].sum().sum()
        else:
            numerator_val = pos_players_df[numerator].sum()
            
        #denominator_val = pos_players_df[denominator].sum() if denominator is not None else stat_config[]
        if denominator_val == 0:
            avg_stat_val = None
        else:
            avg_stat_val = float(numerator_val) / float(denominator_val)
        
        pos_player_avg_dict["{}_{}_{}".format(col_prepend, pos_colname, stat)] = avg_stat_val
    
    return pos_player_avg_dict


In [27]:
output_rows = []
output_data_dir = os.getcwd().replace('/notebooks', '') + '/data/interim'
output_filename = 'full_game_summary_by_team.csv'

for game_link in unique_games:
    print(game_link)
    team_df_subset = team_df[team_df["url"] == game_link]
    teams = team_df_subset["team"].unique()
    
    for idx, team_row in team_df_subset.iterrows():
        team_row = dict(team_row)
        team = team_row["team"]
        opponent = team_row["opponent"]
        
        off_plays = team_row["team_plays"]
        def_plays = team_row["opp_plays"]
        
        
        season = team_row["season"]
        week = team_row["week"]
        player_game_df_subset = indiv_player_game_df[indiv_player_game_df["season"] == season]
        player_game_df_subset = player_game_df_subset[player_game_df_subset["week"].astype(str) == week]
        team_dict = get_player_stats(player_game_df_subset, team, opp_flg = False, off_snaps = off_plays, def_snaps = def_plays)
        opp_dict = get_player_stats(player_game_df_subset, opponent, opp_flg = True, off_snaps = def_plays, def_snaps = off_plays)
        
        team_row.update(team_dict)
        team_row.update(opp_dict)
        output_rows.append(team_row)

output_df = pd.DataFrame(output_rows)
output_df_first_cols = ['url', 'team', 'season', 'week', 'opponent', 'line', 'over_under', 'off_pts', 'def_pts_allowed', 'home_flg', 'winner_flg']
output_df_other_cols = [x for x in list(output_df.columns) if x not in output_df_first_cols]
output_df_cols = output_df_first_cols + output_df_other_cols
output_df = output_df[output_df_cols]
full_path = os.path.join(output_data_dir, output_filename)
output_df.to_csv(full_path, index = False)

/boxscores/201209050nyg.htm
/boxscores/201209090chi.htm
/boxscores/201209090nyj.htm
/boxscores/201209090nor.htm
/boxscores/201209090oti.htm
/boxscores/201209090det.htm
/boxscores/201209090cle.htm
/boxscores/201209090htx.htm
/boxscores/201209090kan.htm
/boxscores/201209090min.htm
/boxscores/201209090crd.htm
/boxscores/201209090gnb.htm
/boxscores/201209090tam.htm
/boxscores/201209090den.htm
/boxscores/201209100rav.htm
/boxscores/201209100rai.htm
/boxscores/201209130gnb.htm
/boxscores/201209160buf.htm
/boxscores/201209160nwe.htm
/boxscores/201209160mia.htm
/boxscores/201209160cin.htm
/boxscores/201209160clt.htm
/boxscores/201209160phi.htm
/boxscores/201209160jax.htm
/boxscores/201209160nyg.htm
/boxscores/201209160car.htm
/boxscores/201209160ram.htm
/boxscores/201209160sea.htm
/boxscores/201209160sdg.htm
/boxscores/201209160pit.htm
/boxscores/201209160sfo.htm
/boxscores/201209170atl.htm
/boxscores/201209200car.htm
/boxscores/201209230cle.htm
/boxscores/201209230nor.htm
/boxscores/201209230

/boxscores/201309150crd.htm
/boxscores/201309150rai.htm
/boxscores/201309150nyg.htm
/boxscores/201309150sea.htm
/boxscores/201309160cin.htm
/boxscores/201309190phi.htm
/boxscores/201309220nwe.htm
/boxscores/201309220dal.htm
/boxscores/201309220rav.htm
/boxscores/201309220nor.htm
/boxscores/201309220oti.htm
/boxscores/201309220cin.htm
/boxscores/201309220was.htm
/boxscores/201309220car.htm
/boxscores/201309220min.htm
/boxscores/201309220mia.htm
/boxscores/201309220nyj.htm
/boxscores/201309220sfo.htm
/boxscores/201309220sea.htm
/boxscores/201309220pit.htm
/boxscores/201309230den.htm
/boxscores/201309260ram.htm
/boxscores/201309290buf.htm
/boxscores/201309290kan.htm
/boxscores/201309290htx.htm
/boxscores/201309290tam.htm
/boxscores/201309290det.htm
/boxscores/201309290jax.htm
/boxscores/201309290cle.htm
/boxscores/201309290min.htm
/boxscores/201309290oti.htm
/boxscores/201309290den.htm
/boxscores/201309290rai.htm
/boxscores/201309290sdg.htm
/boxscores/201309290atl.htm
/boxscores/201309300

/boxscores/201409280pit.htm
/boxscores/201409280rai.htm
/boxscores/201409280sdg.htm
/boxscores/201409280min.htm
/boxscores/201409280sfo.htm
/boxscores/201409280dal.htm
/boxscores/201409290kan.htm
/boxscores/201410020gnb.htm
/boxscores/201410050oti.htm
/boxscores/201410050jax.htm
/boxscores/201410050phi.htm
/boxscores/201410050nor.htm
/boxscores/201410050car.htm
/boxscores/201410050nyg.htm
/boxscores/201410050dal.htm
/boxscores/201410050clt.htm
/boxscores/201410050det.htm
/boxscores/201410050den.htm
/boxscores/201410050sfo.htm
/boxscores/201410050sdg.htm
/boxscores/201410050nwe.htm
/boxscores/201410060was.htm
/boxscores/201410090htx.htm
/boxscores/201410120cle.htm
/boxscores/201410120nyj.htm
/boxscores/201410120oti.htm
/boxscores/201410120mia.htm
/boxscores/201410120tam.htm
/boxscores/201410120min.htm
/boxscores/201410120buf.htm
/boxscores/201410120cin.htm
/boxscores/201410120rai.htm
/boxscores/201410120crd.htm
/boxscores/201410120atl.htm
/boxscores/201410120sea.htm
/boxscores/201410120

/boxscores/201510180jax.htm
/boxscores/201510180min.htm
/boxscores/201510180buf.htm
/boxscores/201510180oti.htm
/boxscores/201510180pit.htm
/boxscores/201510180sea.htm
/boxscores/201510180sfo.htm
/boxscores/201510180gnb.htm
/boxscores/201510180clt.htm
/boxscores/201510190phi.htm
/boxscores/201510220sfo.htm
/boxscores/201510250jax.htm
/boxscores/201510250ram.htm
/boxscores/201510250nwe.htm
/boxscores/201510250mia.htm
/boxscores/201510250oti.htm
/boxscores/201510250det.htm
/boxscores/201510250clt.htm
/boxscores/201510250kan.htm
/boxscores/201510250was.htm
/boxscores/201510250sdg.htm
/boxscores/201510250nyg.htm
/boxscores/201510250car.htm
/boxscores/201510260crd.htm
/boxscores/201510290nwe.htm
/boxscores/201511010kan.htm
/boxscores/201511010chi.htm
/boxscores/201511010ram.htm
/boxscores/201511010pit.htm
/boxscores/201511010rav.htm
/boxscores/201511010cle.htm
/boxscores/201511010htx.htm
/boxscores/201511010atl.htm
/boxscores/201511010nor.htm
/boxscores/201511010rai.htm
/boxscores/201511010

/boxscores/201610300cin.htm
/boxscores/201610300car.htm
/boxscores/201610300cle.htm
/boxscores/201610300buf.htm
/boxscores/201610300tam.htm
/boxscores/201610300clt.htm
/boxscores/201610300nor.htm
/boxscores/201610300htx.htm
/boxscores/201610300den.htm
/boxscores/201610300atl.htm
/boxscores/201610300dal.htm
/boxscores/201610310chi.htm
/boxscores/201611030tam.htm
/boxscores/201611060rav.htm
/boxscores/201611060nyg.htm
/boxscores/201611060mia.htm
/boxscores/201611060kan.htm
/boxscores/201611060cle.htm
/boxscores/201611060min.htm
/boxscores/201611060sfo.htm
/boxscores/201611060ram.htm
/boxscores/201611060gnb.htm
/boxscores/201611060sdg.htm
/boxscores/201611060rai.htm
/boxscores/201611070sea.htm
/boxscores/201611100rav.htm
/boxscores/201611130car.htm
/boxscores/201611130oti.htm
/boxscores/201611130was.htm
/boxscores/201611130jax.htm
/boxscores/201611130tam.htm
/boxscores/201611130nor.htm
/boxscores/201611130nyj.htm
/boxscores/201611130phi.htm
/boxscores/201611130sdg.htm
/boxscores/201611130

/boxscores/201711120jax.htm
/boxscores/201711120was.htm
/boxscores/201711120det.htm
/boxscores/201711120oti.htm
/boxscores/201711120clt.htm
/boxscores/201711120tam.htm
/boxscores/201711120ram.htm
/boxscores/201711120atl.htm
/boxscores/201711120sfo.htm
/boxscores/201711120den.htm
/boxscores/201711130car.htm
/boxscores/201711160pit.htm
/boxscores/201711190chi.htm
/boxscores/201711190min.htm
/boxscores/201711190nyg.htm
/boxscores/201711190gnb.htm
/boxscores/201711190cle.htm
/boxscores/201711190nor.htm
/boxscores/201711190htx.htm
/boxscores/201711190mia.htm
/boxscores/201711190sdg.htm
/boxscores/201711190den.htm
/boxscores/201711190rai.htm
/boxscores/201711190dal.htm
/boxscores/201711200sea.htm
/boxscores/201711230det.htm
/boxscores/201711230dal.htm
/boxscores/201711230was.htm
/boxscores/201711260clt.htm
/boxscores/201711260kan.htm
/boxscores/201711260phi.htm
/boxscores/201711260atl.htm
/boxscores/201711260nyj.htm
/boxscores/201711260cin.htm
/boxscores/201711260nwe.htm
/boxscores/201711260

In [24]:
for col in output_df.columns:
    print(col)

def_pts_allowed
favorite_flg
home_flg
line
off_pts
opp_db_def_int_sum
opp_db_fumbles_forced_sum
opp_db_sacks_sum
opp_db_snaps_sum
opp_db_tackles_assists_sum
opp_db_tackles_per_snap
opp_db_tackles_solo_sum
opp_dl_fumbles_forced_sum
opp_dl_sacks_sum
opp_dl_snaps_sum
opp_dl_tackles_assists_sum
opp_dl_tackles_per_snap
opp_dl_tackles_solo_sum
opp_drives
opp_fgs
opp_first_downs
opp_fourth_down_conv
opp_fumbles
opp_fumbles_lost
opp_k_fg_pct
opp_k_fga_sum
opp_k_fgm_sum
opp_k_xpa_sum
opp_k_xpm_sum
opp_lb_fumbles_forced_sum
opp_lb_sacks_sum
opp_lb_snaps_sum
opp_lb_tackles_assists_sum
opp_lb_tackles_per_snap
opp_lb_tackles_solo_sum
opp_net_pass_yds
opp_p_punt_sum
opp_p_punt_yds_per_punt_sum
opp_pass_att
opp_pass_comp
opp_pass_int
opp_pass_tds
opp_pass_yds
opp_penalties
opp_penalty_yds
opp_plays
opp_punts
opp_qb_fumbles_sum
opp_qb_pass_att_sum
opp_qb_pass_cmp_sum
opp_qb_pass_int_sum
opp_qb_pass_sacked_sum
opp_qb_pass_yds_sum
opp_qb_rush_att_sum
opp_qb_rush_long_sum
opp_qb_rush_td_sum
opp_qb_rush_y